In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [5]:
df = spark.read.csv('SMSSpamCollection',inferSchema=True,sep='\t')

In [6]:
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [7]:
df = df.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [10]:
from pyspark.sql.functions import length

In [11]:
df = df.withColumn('length',length(df['text']))

In [12]:
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [13]:
df.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [15]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [16]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')

In [17]:
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')

In [18]:
count_vec = CountVectorizer(inputCol='stop_token',outputCol='count_vec')

In [19]:
idf = IDF(inputCol='count_vec',outputCol='tf_idf')

In [20]:
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [22]:
from pyspark.ml.feature import VectorAssembler

In [23]:
final_df = VectorAssembler(inputCols=['tf_idf','label'],outputCol='features')

In [25]:
from pyspark.ml.classification import NaiveBayes

In [26]:
nb = NaiveBayes()

In [28]:
from pyspark.ml import Pipeline

In [29]:
data_prep_pipeline = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,final_df])

In [31]:
cleaner = data_prep_pipeline.fit(df)

In [33]:
clean_data = cleaner.transform(df)

In [36]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'count_vec',
 'tf_idf',
 'features']

In [37]:
clean_data = clean_data.select('label','features')

In [40]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [41]:
train_data,test_data = clean_data.randomSplit([0.7,0.3])

In [42]:
spam_detector = nb.fit(train_data)

In [43]:
test_results = spam_detector.transform(test_data)

In [44]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-658.92467796144...|[1.0,4.1183735455...|       0.0|
|  0.0|(13424,[0,1,2,13,...|[-511.75752709700...|[1.0,3.3252433026...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-696.61945976279...|[1.0,1.1566806583...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-834.80182351772...|[1.0,3.3028974184...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-170.69032650863...|[1.0,7.7593824470...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-674.52298364345...|[1.0,1.5012215667...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-734.26979420610...|[1.0,3.8877817053...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-632.03861018862...|[1.0,8.1786831755...|       0.0|
|  0.0|(13424,[0,1,24,31...|[-292.58677472712...|[1.0,1.7152016413...|       0.0|
|  0.0|(13424,[0

In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [46]:
acc_eval = MulticlassClassificationEvaluator()

In [47]:
acc = acc_eval.evaluate(test_results)

In [48]:
print('ACC of NB Model')
print(acc)

ACC of NB Model
0.9173964513325802
